In [2]:
from typing import re
! pip install simpy

You should consider upgrading via the 'C:\Users\ismae\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [162]:
from scipy.stats import bernoulli

valor = bernoulli.rvs(0.15)
valor

1

In [163]:
#Importacion de librerias necesarias
import collections
import random
import simpy
import matplotlib.pyplot as plt
import numpy as np
import statistics as stats
from collections import Counter
from scipy.stats import bernoulli
print('Librerias importadas')

Librerias importadas


In [96]:
#DECLARACION DE VARIABLES
CARRIL_LIVIANO=2
CARRIL_PESADO=1
TIEPO_LLEGADA=3
TIEMPO_REVISION=20 #MIN
T_SIMULACION=30*24*60
tipoVehiculo= ["pesados", "livianos"]
carrosLivianos=[]
carrosPesado=[]

In [100]:
class CuencaAire(object):

    def __init__(self,env,carril_liviano,carril_pesado,tiempo_revision):
        self.env=env
        self.carril_li = simpy.Resource(env,carril_liviano)
        self.carril_pe=simpy.Resource(env,carril_pesado)
        self.tiempo_revision=tiempo_revision
        self.tiempo=0

    def llega_vehiculo(self,automovil):
        print('Llega vehiculo: %s  de tipo :%s ' % (automovil))

    def estacionarse(self,automovil):
          yield self.env.timeout(random.randint(3,5))
          print('El vehiculo: %s de demoro en estacionarse un tiempo de: %s ' % (automovil, random.randint(3,5)))

    def obtener_turno(self,automovil):
        yield self.env.timeout(random.randint(1,3))
        print('El vehiculo: %s obtubo un turno en un tiempo de: %s ' % (automovil, random.randint(1,3)))

    def Pagar_matricula(self,env,nombre,tipo):
        yield self.env.timeout(random.randint(5,10))
        print('Se paga el turno para el vehiculo: %s en un tiempo de: %s ' % (nombre, random.randint(5,10)))

    def revision(self, automovil):
        valor_x = random.normalvariate(20, 10)
        while valor_x < 0:
            valor_x = random.normalvariate(20, 10)

        yield self.env.timeout(valor_x)
        print('Se realizo el revisado del vehiculo : %s ' % (automovil))

    def pegar_Sello(self, automovil):
        yield self.env.timeout(random.randint(1,3))
        print('El vehiculo paso la prueba y se procedde a pegar el sello: %s ' % (automovil))

    def salida(self, automovil):
        yield self.env.timeout(random.randint(5,10))
        print('El vehiculo se retira : %s ' % (automovil))


def llegada_vehiculo(env, nombre, tipo, emov):
    print('Llega vehiculo: %s a la hora %.2f.' % (nombre, env.now))

    print('El Vehiculo : %s empieza a estacionarse a la hora %.2f.' % (nombre, env.now))
    yield env.process(emov.estacionarse(nombre))
    print('El Vehiculo : %s se estaciona a la hora  %.2f.' % (nombre, env.now))

    print('El Vehiculo : %s procede a sacar un turno a la hora %.2f.' % (nombre, env.now))
    yield env.process(emov.estacionarse(nombre))
    print('El Vehiculo : %s Obtiene un turno  a la hora  %.2f.' % (nombre, env.now))

    print('El Vehiculo : %s procede a pagar la matricula a la hora %.2f.' % (nombre, env.now))
    yield env.process(emov.estacionarse(nombre))
    print('El Vehiculo : %s pago su matricula en un tiempo de  %.2f.' % (nombre, env.now))


    if(tipo=="livianos"):
        print("Vehiculo Liviano se procede a revisar disponibilidad de carril")
        with emov.carril_li.request() as carril:
            yield carril
            print('---El Vehiculo: %s Ingresa a revision a la hora %.2f.' % (nombre, env.now))
            yield env.process(emov.revision(nombre))
            print('----El Vehiculo: %s Finalizo la revision a la hora %.2f.' % (nombre, env.now))
            carrosLivianos.append([nombre,env.now])
            #carril.processed()

            print('El Vehiculo : %s procede a recibir su stiker %.2f.' % (nombre, env.now))
            yield env.process(emov.pegar_Sello(nombre))
            print('El Vehiculo : %s pego su stiker en un tiempo de  %.2f.' % (nombre, env.now))

            print('El Vehiculo : %s procede a retirarse %.2f.' % (nombre, env.now))
            yield env.process(emov.pegar_Sello(nombre))
            print('El Vehiculo : %s se retiro en un tiempo de %.2f.' % (nombre, env.now))


            print("************************",carrosLivianos)


    else:
        print("Vehiculo Pesado se procede a revisar disponibilidad de carril")
        with emov.carril_pe.request() as carrilPe:
            yield carrilPe
            print('---El Vehiculo: %s Ingresa a revision a la hora %.2f.' % (nombre, env.now))
            yield env.process(emov.revision(nombre))
            print('----El Vehiculo: %s Finalizo la revision a la hora %.2f.' % (nombre, env.now))

            carrosPesado.append([nombre,env.now])

            print('El Vehiculo : %s procede a recibir su stiker %.2f.' % (nombre, env.now))
            yield env.process(emov.pegar_Sello(nombre))
            print('El Vehiculo : %s pego su stiker en un tiempo de  %.2f.' % (nombre, env.now))

            print('El Vehiculo : %s procede a retirarse %.2f.' % (nombre, env.now))
            yield env.process(emov.pegar_Sello(nombre))
            print('El Vehiculo : %s se retiro en un tiempo de %.2f.' % (nombre, env.now))





def ejecutar_simulacion(env,carril_liviano,carril_pesado,tiempo_revision):
    emov = CuencaAire(env,carril_liviano,carril_pesado,tiempo_revision)
    valor = bernoulli.rvs(0.15)
    valor
    if(valor==1):
        tipo_vehiculo="pesados"
    else:
        tipo_vehiculo="livianos"

    for i in range(5):



        tipo_vehiculo = random.choice(tipoVehiculo)
        print(tipo_vehiculo)
        env.process(llegada_vehiculo(env, 'Vehiculo-%d'%(i+1), tipo_vehiculo, emov))

    while True:
        tipo_vehiculo = random.choices(tipoVehiculo)
        yield env.timeout(3)
        i+=1
        env.process(llegada_vehiculo(env, 'Vehiculo-%d'%(i+1), tipo_vehiculo, emov))

print('Revisión de Vehículos')
random.seed(77)


env=simpy.Environment()
env.process(ejecutar_simulacion(env, CARRIL_LIVIANO, CARRIL_PESADO, TIEMPO_REVISION))

# Ejecutamos el proceso durante el tiempo de simulacion
env.run(until = T_SIMULACION)



Revisión de Vehículos
livianos
livianos
pesados
pesados
pesados
Llega vehiculo: Vehiculo-1 a la hora 0.00.
El Vehiculo : Vehiculo-1 empieza a estacionarse a la hora 0.00.
Llega vehiculo: Vehiculo-2 a la hora 0.00.
El Vehiculo : Vehiculo-2 empieza a estacionarse a la hora 0.00.
Llega vehiculo: Vehiculo-3 a la hora 0.00.
El Vehiculo : Vehiculo-3 empieza a estacionarse a la hora 0.00.
Llega vehiculo: Vehiculo-4 a la hora 0.00.
El Vehiculo : Vehiculo-4 empieza a estacionarse a la hora 0.00.
Llega vehiculo: Vehiculo-5 a la hora 0.00.
El Vehiculo : Vehiculo-5 empieza a estacionarse a la hora 0.00.
Llega vehiculo: Vehiculo-6 a la hora 3.00.
El Vehiculo : Vehiculo-6 empieza a estacionarse a la hora 3.00.
El vehiculo: Vehiculo-1 de demoro en estacionarse un tiempo de: 5 
El Vehiculo : Vehiculo-1 se estaciona a la hora  3.00.
El Vehiculo : Vehiculo-1 procede a sacar un turno a la hora 3.00.
El vehiculo: Vehiculo-2 de demoro en estacionarse un tiempo de: 5 
El vehiculo: Vehiculo-3 de demoro en es

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [103]:
print("Carros atendidos por el carril liviano",len(carrosLivianos))
print("Carros atendidos por el carril pesados",len(carrosPesado))


print("Porcentaje de vehiculos aprobados")


print(carrosLivianos)
print("******************")
print(carrosPesado)

Carros atendidos por el carril liviano 6
Carros atendidos por el carril pesados 1753
Porcentaje de vehiculos aprobados
[['Vehiculo-2', 31.277736620914528], ['Vehiculo-1', 44.15084156799803], ['Vehiculo-2', 31.277736620914528], ['Vehiculo-1', 44.15084156799803], ['Vehiculo-2', 31.277736620914528], ['Vehiculo-1', 44.15084156799803]]
******************
[['Vehiculo-3', 34.793600795774445], ['Vehiculo-4', 64.41005734225284], ['Vehiculo-5', 106.67474115269073], ['Vehiculo-6', 124.93845395469388], ['Vehiculo-7', 145.08425885983957], ['Vehiculo-8', 169.37844432921074], ['Vehiculo-9', 190.94389185910555], ['Vehiculo-10', 203.121101116516], ['Vehiculo-11', 218.10125801096612], ['Vehiculo-12', 238.02120288231077], ['Vehiculo-13', 261.8713826168204], ['Vehiculo-14', 282.535805245396], ['Vehiculo-15', 301.2118645665353], ['Vehiculo-16', 335.2537650579099], ['Vehiculo-17', 351.8050395422512], ['Vehiculo-18', 381.3564597642526], ['Vehiculo-19', 408.22991647853974], ['Vehiculo-21', 432.76267776458127]

In [140]:
dias_en_minutos = 24*60
segundos = list([x*24*60 for x in range(31)])
segundos.pop(0)

dia_actual = segundos[0]
resultados = dict()
lista = []

for i in carrosPesado:
    if i[1] <= dia_actual:
        lista.append(i[1])
    else:
        resultados[dia_actual] = lista
        dia_actual = segundos[segundos.index(dia_actual)+1]
        lista = list()


rst = {x: len(y) for x, y in resultados.items()}
from pandas import DataFrame
import plotly.express as ex

df = DataFrame(rst.items())
df.columns = ['dia', 'cantidad']

grafica = ex.bar(df, 'dia', 'cantidad')
grafica.show()